# Wix Test - BI Developer  - Daniel Levkovitz

### Import Packages

In [1]:
import mysql.connector
import pandas as pd
import numpy as np
import requests
import sqlalchemy
import json

### Import Credentials for MySQL

I stored the connection details in a separate file as described in the test.

In [2]:
credentials = open("C:\\Users\\daniel_l\\Documents\\credentials_mysql.json", encoding="ISO-8859-1")

In [3]:
credentials = json.load(credentials)

### Log In to the Database

In [40]:
connect_url = sqlalchemy.engine.URL.create(drivername='mysql+mysqlconnector',username=credentials['User'],port=credentials['Port'],password=credentials['Password'],host=credentials['Host'],database=credentials['Database'])

In [41]:
engine = sqlalchemy.create_engine(connect_url)

### Import Data from https://randomuser.me/

In [7]:
results = requests.get("https://randomuser.me/api/?format=JSON&results=4500&noinfo")

In [8]:
results = results.json()['results']

### Requirement 1 - Create dataset of 4500 users

In [9]:
df = pd.DataFrame(pd.json_normalize(results))

### Requirement 2 - Separate the dataset into 2 gender datasets (male and female)

In [10]:
df_male = df[df['gender']=='male']
df_female = df[df['gender']=='female']

#### Store each one of the datasets in a separated MySQL table

In [11]:
df_male.to_sql('DANIEL_LEVKOVITZ_test_male', engine, index=False)

2253

In [12]:
df_female.to_sql('DANIEL_LEVKOVITZ_test_female', engine, index=False)

2247

### Requirement 3 - Split the dataset into 10 subsets by dob.age column in groups of 10

In [13]:
def age_band(num):
    for i in range(1, 11):
        if num < 10*i :  return (i-1) * 10

In [14]:
df_age_dicts={}
for i in range(1,11):
    df_age_dicts["df_{0}".format(i*10)] = df.loc[df['dob.age'].apply(age_band)==i*10]

### Requirement 4 - Store each one of the subsets in MySQL

In [15]:
for i in range(1,11):
    df_age_dicts["df_{0}".format(i*10)].to_sql('DANIEL_LEVKOVITZ_test_{0}'.format(i), engine, index=False)

### Requirement 5 - Write a SQL query that will return the top 20 last registered males and females from each one of the gender tables you created in 2 and save it in MySQL

In [20]:
top_registered = engine.execute("SELECT * FROM interview.DANIEL_LEVKOVITZ_test_male ORDER BY `registered.date` DESC LIMIT 20").fetchall() + engine.execute("SELECT * FROM interview.DANIEL_LEVKOVITZ_test_female ORDER BY `registered.date` DESC LIMIT 20").fetchall() 

In [21]:
df_top_registered = pd.DataFrame(columns=df.columns, data=top_registered)

In [22]:
df_top_registered.to_sql('DANIEL_LEVKOVITZ_test_20', engine, index=False)

40

### Requirement 6 - Create a dataset that combines data from DANIEL_LEVKOVITZ__test_20 and data from the DANIEL_LEVKOVITZ_test_5 table. 
### Make sure each row is presented only once, and there is no multiplication of data

In [42]:
combined_data_q6 = engine.execute("SELECT  * FROM DANIEL_LEVKOVITZ_test_20 UNION SELECT * FROM DANIEL_LEVKOVITZ_test_5").fetchall()

In [24]:
df_combined_data_q6 = pd.DataFrame(columns = df.columns, data = combined_data_q6)

#### Create JSON from the mentioned dataset and store it locally as first.json

In [25]:
df_combined_data_q6.to_json('first.json', indent=True,orient = 'index')

### Requirement 7 - Create a dataset that combines data from DANIEL_LEVKOVITZ__test_20 and data from the DANIEL_LEVKOVITZ_test_5 table.
### In case the same rows are presented in 2 datasets both rows are supposed to be presented.

In [36]:
combined_data_q7 = engine.execute("SELECT  * FROM DANIEL_LEVKOVITZ_test_20 UNION ALL SELECT * FROM DANIEL_LEVKOVITZ_test_2").fetchall()

In [37]:
df_combined_data_q7 = pd.DataFrame(columns = df.columns, data = combined_data_q7)

#### Create json from the mentioned dataset and store it locally as second.json

In [39]:
df_combined_data_q7.to_json('second.json', indent=True,orient = 'index')